# Loops and iteration

## Issue analysis
The programs we have written so far can only perform a limited number of maximum steps. This means that they will, invariably, terminate after a given number of steps. For example, a program such as `x := 0; y := 2; done` will perform exactly two steps (ignoring the `;` statements and the management of `done; J`).

<img src="images/terminating_traces.png" alt="Terminating traces" style="width: 400px;"/>

Even in the presence of conditional statements, although the exact number of steps is not precisely known in advance, we still know what the upper bound is: the longest sequence of decisions in the conditionals. For example, `x := int(input()); if x > 0 then { x := x * 2; x := x + 1 } else { x := -x; x := x * 2; x := x + 1 }; done` can perform, at most, the evaluation of four statements (when `x` is smaller or equal to zero), whereas the shortest path to `done` is only three statements. Thus, a conditional statement makes it harder, or even impossible, to determine the _exact_ number of steps, but we still know how many steps _at most_.

<img src="images/terminating_conditional_traces.png" alt="Terminating conditional traces" style="width: 400px;"/>

The issue we face is that programs are a way to solve a _problem_. A problem is a form of pathfinding in the network of all possible states represented by the computer. We want to **connect a given initial state (the input of the problem) to a given final state (the output to the problem)**. This path we are looking for, connecting a problem input to its output, must be a *program*: we do not want to manually find the connection ourselves (that would be akin to a mathematician proving that the result is indeed formally acceptable). Instead, we want to build a program that does it for us.

<img src="images/solvable_io_pairs.png" alt="Solvable input/output pairs" style="width: 400px;"/>

Moreover, it might be that a program solving our problem does not exist: there exist some problems so complex that not even a very powerful computer could solve them (this is, unfortunately, proven mathematically: there do exist some problems that, no matter the computer or the programming language used, cannot be solved). This would mean that the input and output are not connected by any path:

<img src="images/unsolvable_io_pairs.png" alt="Unsolvable input/output pairs" style="width: 400px;"/>

In such a case, no matter how clever we are, we will not be able to produce a program to solve this problem. Fortunately, computers are *very* powerful, so there are not so many problems that cannot be solved in any way. The language, especially in the beginning, will be the most limiting factor. For example, let us consider the following simple problem: _build a program that reads as input a number, and builds a string with as many asterisks as the input number_. The input/output pairs look like:

 Input | Output
:-------------------------:|:----------------------------:
`<P, ` $\{ \text{input} := \{ \text{next} := \text{"0"}, \dots \} \}$ `>` | `<done, ` $\{ \text{input} := \{\}, \text{s} := \text{""} \}$ `>` |
`<P, ` $\{ \text{input} := \{ \text{next} := \text{"1"}, \dots \} \}$ `>` | `<done, ` $\{ \text{input} := \{\}, \text{s} := \text{"*"} \}$ `>` |
`<P, ` $\{ \text{input} := \{ \text{next} := \text{"2"}, \dots \} \}$ `>` | `<done, ` $\{ \text{input} := \{\}, \text{s} := \text{"**"} \}$ `>` |

where `P` is the program we are looking for.

We might attempt to solve this with a series of conditional statements, such as:

```
l := int(input())
if l <= 0 then
  s := ""
elif l == 1 then
  s := "*"
elif l == 2 then
  s := "**"
elif l == 3 then
  s := "***"
...
else
  s := "too long, sorry :*("
```

While this is clearly a reasonable attempt, because it covers at least a subset of all input/output combinations, it does not cover all of them. Moreover, since there exists an infinite number of such pairs, and our program only covers a finite subset, then there will be infinite instances of the problem that we are not solving with this program. This means that we are falling very short of the mark. Fortunately, we can define a simple, yet powerful extension of our programming language that solves this very issue.


## Loops
Let us observe the structure of the program we have just written: the code contains a "repeating pattern" that we want to identify and exploit. We begin by giving a name (right-aligned) to the various sub-programs inside the conditionals:

```
l := int(input())
if l <= 0 then
  s := ""                       P0
elif l == 1 then
  s := "*"                      P1
elif l == 2 then
  s := "**"                     P2
elif l == 3 then
  s := "***"                    P3
...                            ...
else
  s := "too long, sorry :*("
```

There is a relationship between subsequent code blocks such as $P_1$ and $P_0$, that is we can build $P_1$ by running $P_0$ followed by a statement that adds an asterisk to `s`, or in short:

$P_1 = P_0$ `; s := s + "*"`

If we replace in the equation above the definitions of $P_0$ and $P_1$ (that is, the corresponding statements from the program written above), we get:

- $P_1 = P_0$ `; s := s + "*"`
- $P_1 = $ `s := ""; s := s + "*"` because $P_0 = $ `s := ""`
- `s := "*"` $=$ `s := ""; s := s + "*"` because $P_1 = $ `s := "*"`

Similarly, we get that $P_2 = P_1$ `; s := s + "*"`, but also $P_3 = P_2$ `; s := s + "*"`, and so on. Instead of repeating `s := s + "*"`, let us give a name to it: $\Delta_P = $ `s := s + "*"`. Then we get the following table of results:

\begin{array}{lcr}
P_1 & = & P_0; \Delta_P \\
P_2 & = & P_1; \Delta_P \\
P_3 & = & P_2; \Delta_P \\
P_4 & = & P_3; \Delta_P \\
    & \vdots &          \\
\end{array}

Let us consider a single instance of this, that is $P_3$. According to the table above, $P_3 = P_2; \Delta_P$. Let us replace the value of $P_2$: $P_3 = P_1; \Delta_P; \Delta_P$. Let us now replace the value of $P_1$: $P_3 = P_0; \Delta_P; \Delta_P; \Delta_P$. This makes perfect sense, as it is stating that, in order to produce a string with three asterisks, we must run the program $P_0; \Delta_P; \Delta_P; \Delta_P$, that is: `s := ""; s := s + "*"; s := s + "*"; s := s + "*"`.

So, in general, given an input `l`, what we really need to do is always:
- evaluate $P_0$ once (`s := ""`);
- evaluate $\Delta_P$ (`s := s + "*"`) repeated `l` times one after another.

We therefore introduce a new construct, the `while` loop, to express this concept. Its syntax, `while C do P`, combines together two pieces of code, which it then coordinates together to perform a unique function:
- `C` is a boolean expression which determines whether or not we should stop right away, and it is called **condition** of the loop;
- `P` is a full-blown sub-program which is run once every time a check on `c` is passed, and it is called **body** of the loop.

The semantics of `while` make use of the semantics of `if` (given that they are already defined: there is no reason to reinvent the wheel for every new programming construct!), but at the same time they extend the program a bit in order to account for the repetition:
- `eval(<while C do P>, S) ` $\rightarrow$ `( <if C then { P; while C do P } else { done }>, S )`.

This semantic rule produces the desired repetition, as long as `C` remains `True` in state `S`: evaluating the `while` loop will require first evaluating an `if` statement; if the condition is `True`, then we will evaluate `P` once, and then the whole `while` loop, unchanged, again. Of course, since `P` does change the state, even though the `while` loop is evaluated again, we hope that eventually the condition `C` will become `False` as a result of the last evaluation of `P`.

Let us take a look at a few simple introductory examples of `while` loops:

```
i := 0
while i < 3 do
  i := i + 1
```

Notice that variable `i` is inexorably working its way "up" towards value `3`, one step at a time. The resulting trace is:

 Statement              |            State   
:----------------------|:----------------------------
 `i := 0; ...`          |    $\{ \}$   
 `while i < 3 do ...` |    $\{ i := 0 \}$   
 `if i < 3 then ...` |    $\{ i := 0 \}$   
 `if 0 < 3 then ...` |    $\{ i := 0 \}$   
 `if True then ...` |    $\{ i := 0 \}$   
 `i := i + 1; while ...` |    $\{ i := 0 \}$   
 `i := 0 + 1; while ...` |    $\{ i := 0 \}$   
 `i := 1; while ...` |    $\{ i := 0 \}$   
 `while i < 3 do ...` |    $\{ i := 1 \}$   
 `if i < 3 then ...` |    $\{ i := 1 \}$   
 `if 1 < 3 then ...` |    $\{ i := 1 \}$   
 `if True then ...` |    $\{ i := 1 \}$   
 `i := i + 1; while ...` |    $\{ i := 1 \}$   
 `i := 1 + 1; while ...` |    $\{ i := 1 \}$   
 `i := 2; while ...` |    $\{ i := 1 \}$   
 `while i < 3 do ...` |    $\{ i := 2 \}$   
 `if i < 3 then ...` |    $\{ i := 2 \}$   
 `if 2 < 3 then ...` |    $\{ i := 2 \}$   
 `if True then ...` |    $\{ i := 2 \}$   
 `i := i + 1; while ...` |    $\{ i := 2 \}$   
 `i := 2 + 1; while ...` |    $\{ i := 2 \}$   
 `i := 3; while ...` |    $\{ i := 2 \}$   
 `while i < 3 do ...` |    $\{ i := 3 \}$   
 `if i < 3 then ...` |    $\{ i := 3 \}$   
 `if 3 < 3 then ...` |    $\{ i := 3 \}$   
 `if False then ...` |    $\{ i := 3 \}$   
 `done` |    $\{ i := 3 \}$   

Another example of a program we might want to study as example is:

```
i := 1
while i < 4 do
  i := i * 2
```

In this case, variable `i` is again working its way "up", but this time by being multiplied by two every time. We could tabulate this in a more succint way by simply tracking the sequence of values of `i` until it gets greater or equal than $4$:

| `i` |
|:---:|
| $1$ |
| $2$ |
| $4$ |

Notice that the same program, with the only difference that variable `i` starts from zero, would never terminate:

```
i := 0
while i < 4 do
  i := i * 2
```

In this case, variable `i` tries again to "work its way up", but this time the multiplication by two has no effect since the value of `i` is zero at the beginning and remains thus zero after the multiplication. We can see this also by tabulating the sequence of values of `i` as we did before:

| `i` |
|:---:|
| $0$ |
| $0$ |
| $0$ |
| $\vdots$ |

Let us now go to back the original example that started us along this path of exploration.

```
l := int(input())
if l <= 0 then
  s := ""               P0
elif l == 1 then
  s := "*"              P1
elif l == 2 then
  s := "**"             P2
elif l == 3 then
  s := "***"            P3
...
```

As we concluded above, each sub-program inside an `if` statement can be replaced with $\Delta_P$, that is `s := s + "*"`, repeated as many times as `l`, and starting from $P_0$, that is `s := ""`.

This would lead us to an expanded version of the code above. Despite its verbosity, the repeating structure is now even more evident:

```
l := int(input())
if l <= 0 then
  s := ""
elif l == 1 then
  s := ""
  s := s + "*"
elif l == 2 then
  s := ""
  s := s + "*"
  s := s + "*"
elif l == 3 then
  s := ""
  s := s + "*"
  s := s + "*"
  s := s + "*"
...
```

We can then reduce verbosity by taking advantage of the previous steps instead of repeating them. By removing the `else` clauses, and making the conditions inclusive of the subsequent steps, we get to the more concise formulation: 

```
l := int(input())
s := ""
if l >= 1 then
  s := s + "*"
if l >= 2 then
  s := s + "*"
if l >= 3 then
  s := s + "*"
...
```

We have now explicitly identified the repeating instruction(s), and their starting point. It is now almost trivial to encode this with a `while` loop:

```
l := int(input())
s := ""
i := 0
while i < l do
  s := s + "*"
  i := i + 1
```

Notice that the loop contains of course the real payload of the computation, which consists of repeating $\Delta_P$ `l` times. The loop also requires some additional bookkeeping in the form of variable `i`, which counts how many times we have evaluated $\Delta_P$ so far in order to stop after the right number of iterations.

Let us consider an example where the input is `"2"`, in our condensed tabular notation:

 `i` | `s`
:---:|:---
 $0$ | `""`
 $1$ | `"*"`
 $2$ | `"**"`
 
Evaluation of the program stops at `i` $= 2$ because then the condition `i < l` is false (`2 < 2` $\rightarrow$ `False`).


### Expressive power
The program we have just written is capable of solving all problems where the input is a number, and the output is a string containing as many asterisks as the input number. There are infinite such input/output pairs (one for each natural number), and thus our program consists of infinite paths in the network of all computations. Moreover, these paths have no upper bound in number of steps: as big as the input is, so does the output string become and so many steps does the loop perform.

This is a significant improvement in expressive power, and it shows that our programs are now capable of evaluating to dynamically sized, non-bound traces.

### Nesting of loops and conditionals
As we defined `while`, we made no claim about what the body of the loop is supposed to be: it can be just any arbitrary statement, meaning that any program we can write can also be encapsulated inside a loop. This includes conditional statements. The combination of conditional statements inside loops unlocks many interesting patterns that correspond to "alternating" one or more statements in a loop. As a simple, yet visual example, consider:
    
```
l := int(input())
s := ""
while l > 0 do
  if l % 2 == 0 then
    s := s + "+"
  else
    s := s + "*"
  l := l - 1
```

Again, we will only print the values of `s` and `l` after each iteration for brevity, and assuming an input of `"4"`:

|   `s`  |   `l`  |
|:------|:------:|
| `""`   |  `4`   |
| `"+"`   |  `3`   |
| `"+*"`   |  `2`   |
| `"+*+"`   |  `1`   |
| `"+*+*"`   |  `0`   |

Since we know that every conditional statement can double the number of possible paths that a program may take, and that loops can cover infinite such paths, the combination of a conditional inside a loop grants a gigantic coverage of the network of all possible programs, and with very little code. Consider, for example, the following program template (_template_, because we are not specifying what `C`, `C'`, `P`, and `Q` are exactly):

```
while C do
  if C' then
    P
  else
    Q
```

If we make no assumption on the conditions `C` and `C'`, then we must assume that all possible values are attainable by their evaluation. Each step of the loop makes a decision, leading to two possible paths: one starting with `P`, the other with `Q`. If the loop takes no steps, then we run neither `P` nor `Q`. If the loop takes only one step, then we run either `P` or `Q`. If the loop takes two steps, then we run either `P;P`, `P;Q`, `Q;P`, `Q;Q`. In general, for a loop performing $n$ steps, we get a total of $2^n$ possible resulting programs:

[[DRAW DECISION TREE]]

Therefore, the program above covers all possible sequences of `P` and `Q` statements:
- `P;P;P;P;...`
- `Q;Q;Q;Q;...`
- `P;Q;P;Q;...`
- `P;P;Q;Q;...`
- ...

This means that this program template will cover a lot of possible paths, and even stronger: with relatively few iterations a huge number of paths is covered.
    

### Nesting of loops and loops

For the same reason why loops can contain conditional statements, we can observe that loops can contain other loops. This allows us to easily solve problems in which, at every step of the solution, we need to perform a variable number of steps. This corresponds to a "double partition" of the problem, one solving along one "axis" of the problem and the other solving along another "axis". An example that illustrates this intuition about nested loops solving problems with a "bidimensional" structure is drawing a square of asterisks (`\n` is a special character representing a new line):

```
n = int(input())
i := 0
s := ""
while i < n
  j := 0
  while j < n
    s := s + "*"
    j := j + 1
  s := s + "\n"
  i := i + 1
```

This program, for an input of `"2"`, will produce the following table of results:

 `i`  |  `j` |  `s`
:----:|:----:|:----
 `0`  |  `0` | `""`
 `0`  |  `1` | `"*"`
 `0`  |  `2` | `"**"`
 `1`  |  `2` | `"**\n"`
 `1`  |  `0` | `"**\n"`
 `1`  |  `1` | `"**\n*"`
 `1`  |  `2` | `"**\n**"`
 `2`  |  `2` | `"**\n**\n"`

The end result, `"**\n**\n"`, would appear as:

```
**
**
```

If the input value had been `"3"`, then we would get as result (create the table above for yourself as exercise to verify this):

```
***
***
***
```


We will see many more such examples in the following.